In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Path to your Spark installation's JAR directory



In [2]:
spark_jars_dir = "/home/jisna/hadoop/spark1/spark-3.5.1-bin-hadoop3/jars"

In [3]:
# Initialize SparkSession with Kafka package and PostgreSQL JDBC JAR
spark = SparkSession.builder \
    .appName("KafkaStructuredStreamingExample") \
    .config("spark.jars", f"{spark_jars_dir}/postgresql-42.2.18.jar") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()

24/06/04 14:19:04 WARN Utils: Your hostname, jisna-Inspiron-15-3567 resolves to a loopback address: 127.0.1.1; using 192.168.1.104 instead (on interface wlp1s0)
24/06/04 14:19:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/jisna/hadoop/spark1/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jisna/.ivy2/cache
The jars for the packages stored in: /home/jisna/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bc71b24b-286f-46a3-9fe5-e6b61ab3d5b0;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 1774ms :: artifacts dl 69ms
	:: 

24/06/04 14:19:18 INFO SparkContext: Added file file:///home/jisna/.ivy2/jars/org.apache.hadoop_hadoop-client-runtime-3.3.4.jar at file:///home/jisna/.ivy2/jars/org.apache.hadoop_hadoop-client-runtime-3.3.4.jar with timestamp 1717490955556
24/06/04 14:19:18 INFO Utils: Copying /home/jisna/.ivy2/jars/org.apache.hadoop_hadoop-client-runtime-3.3.4.jar to /tmp/spark-0188711a-8359-42db-9f61-2a377e594f01/userFiles-7e404924-bddd-44b9-ba53-63e3216f0c6e/org.apache.hadoop_hadoop-client-runtime-3.3.4.jar
24/06/04 14:19:19 INFO SparkContext: Added file file:///home/jisna/.ivy2/jars/org.lz4_lz4-java-1.8.0.jar at file:///home/jisna/.ivy2/jars/org.lz4_lz4-java-1.8.0.jar with timestamp 1717490955556
24/06/04 14:19:19 INFO Utils: Copying /home/jisna/.ivy2/jars/org.lz4_lz4-java-1.8.0.jar to /tmp/spark-0188711a-8359-42db-9f61-2a377e594f01/userFiles-7e404924-bddd-44b9-ba53-63e3216f0c6e/org.lz4_lz4-java-1.8.0.jar
24/06/04 14:19:19 INFO SparkContext: Added file file:///home/jisna/.ivy2/jars/org.xerial.snapp

In [4]:
# Define the schema for the JSON data
schema = StructType([
    StructField("name", StringType(), True),
    StructField("id", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("place", StringType(), True),
    StructField("country", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone number", StringType(), True)
])

In [5]:
# Create DataFrame representing the stream of input lines from Kafka
df = spark.readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "mytopic") \
  .option("startingOffsets", "earliest") \
  .load()

24/06/04 14:19:22 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/06/04 14:19:22 INFO SharedState: Warehouse path is 'file:/home/jisna/anaconda/spark-warehouse'.


In [6]:
# Convert the value column from binary to string and log raw data from Kafka
raw_df = df.selectExpr("CAST(value AS STRING)")
raw_query = raw_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

24/06/04 14:19:29 INFO StateStoreCoordinatorRef: Registered StateStoreCoordinator endpoint
24/06/04 14:19:29 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-175ac254-d402-4f00-a51d-b0c426533219. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/04 14:19:29 INFO ResolveWriteToStream: Checkpoint root file:///tmp/temporary-175ac254-d402-4f00-a51d-b0c426533219 resolved to file:/tmp/temporary-175ac254-d402-4f00-a51d-b0c426533219.
24/06/04 14:19:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/06/04 14:19:29 INFO CheckpointFileManager: Writing atomically to file:/tmp/temporary-175ac254-d402-4f00-a51d-b0c426533219/metadata using temp file file:/tmp/temporary-175ac254-d402-4f00-a

In [7]:
# Parse the JSON data and apply the schema
value_df = raw_df.select(from_json(col("value"), schema).alias("data"))

24/06/04 14:19:30 INFO MicroBatchExecution: Reading table [org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaTable@3b74460c] from DataSourceV2 named 'kafka' [org.apache.spark.sql.kafka010.KafkaSourceProvider@5ab6c04f]
24/06/04 14:19:30 INFO OffsetSeqLog: BatchIds found from listing: 
24/06/04 14:19:30 INFO OffsetSeqLog: BatchIds found from listing: 
24/06/04 14:19:30 INFO MicroBatchExecution: Starting new streaming query.
24/06/04 14:19:30 INFO MicroBatchExecution: Stream started from {}


In [8]:
# Select individual fields from the parsed JSON
ex_df = value_df.selectExpr(
    "data.name as Name",
    "data.id as Id",
    "data.age as Age",
    "data.gender as Gender",
    "data.place as Place",
    "data.country as Country",
    "data.email as Email",
    "data.`phone number` as Phone"
)

In [9]:
# Define PostgreSQL connection properties
jdbc_url = "jdbc:postgresql://localhost:5432/project_postgress"
jdbc_properties = {
    "user": "jisna",
    "password": "465390",
    "driver": "org.postgresql.Driver"
}

In [ ]:
# Function to write to PostgreSQL
def write_to_postgres(df, epoch_id):
    df.write.jdbc(url=jdbc_url, table="kafka_data", mode="append", properties=jdbc_properties)

# Write the stream to PostgreSQL
query = ex_df.writeStream \
    .foreachBatch(write_to_postgres) \
    .outputMode("append") \
    .start()

query.awaitTermination()

24/06/04 14:19:30 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-462c3203-d908-4b3f-a057-a64297b201f9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/04 14:19:30 INFO ResolveWriteToStream: Checkpoint root file:///tmp/temporary-462c3203-d908-4b3f-a057-a64297b201f9 resolved to file:/tmp/temporary-462c3203-d908-4b3f-a057-a64297b201f9.
24/06/04 14:19:30 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/06/04 14:19:30 INFO CheckpointFileManager: Writing atomically to file:/tmp/temporary-462c3203-d908-4b3f-a057-a64297b201f9/metadata using temp file file:/tmp/temporary-462c3203-d908-4b3f-a057-a64297b201f9/.metadata.f6960745-f695-4f35-9f46-71286b398e3b.tmp
24/06/04 14:19:30 INFO 

24/06/04 14:19:33 INFO KafkaOffsetReaderAdmin: Partitions added: Map()
24/06/04 14:19:33 INFO KafkaOffsetReaderAdmin: Partitions added: Map()
24/06/04 14:19:34 INFO KafkaOffsetReaderAdmin: Partitions added: Map()
24/06/04 14:19:34 INFO KafkaOffsetReaderAdmin: Partitions added: Map()
24/06/04 14:19:34 INFO KafkaOffsetReaderAdmin: Partitions added: Map()
24/06/04 14:19:34 INFO KafkaOffsetReaderAdmin: Partitions added: Map()
24/06/04 14:19:34 INFO KafkaOffsetReaderAdmin: Partitions added: Map()
24/06/04 14:19:34 INFO KafkaOffsetReaderAdmin: Partitions added: Map()
24/06/04 14:19:34 INFO KafkaOffsetReaderAdmin: Partitions added: Map()
24/06/04 14:19:34 INFO KafkaOffsetReaderAdmin: Partitions added: Map()
24/06/04 14:19:35 INFO CodeGenerator: Code generated in 503.75603 ms
24/06/04 14:19:35 INFO CodeGenerator: Code generated in 525.58027 ms
24/06/04 14:19:35 INFO WriteToDataSourceV2Exec: Start processing data source write support: MicroBatchWrite[epoch: 0, writer: ConsoleWriter[numRows=20, 

24/06/04 14:19:37 INFO AppInfoParser: Kafka version: 3.4.1
24/06/04 14:19:37 INFO AppInfoParser: Kafka commitId: 8a516edc2755df89
24/06/04 14:19:37 INFO AppInfoParser: Kafka startTimeMs: 1717490977109
24/06/04 14:19:37 INFO AppInfoParser: Kafka version: 3.4.1
24/06/04 14:19:37 INFO AppInfoParser: Kafka commitId: 8a516edc2755df89
24/06/04 14:19:37 INFO AppInfoParser: Kafka startTimeMs: 1717490977115
24/06/04 14:19:37 INFO KafkaConsumer: [Consumer clientId=consumer-spark-kafka-source-6e653502-7bdf-427e-ad03-8b6cb76d68c4--1298849648-executor-2, groupId=spark-kafka-source-6e653502-7bdf-427e-ad03-8b6cb76d68c4--1298849648-executor] Assigned to partition(s): mytopic-1
24/06/04 14:19:37 INFO KafkaConsumer: [Consumer clientId=consumer-spark-kafka-source-6e653502-7bdf-427e-ad03-8b6cb76d68c4--1298849648-executor-1, groupId=spark-kafka-source-6e653502-7bdf-427e-ad03-8b6cb76d68c4--1298849648-executor] Assigned to partition(s): mytopic-0
24/06/04 14:19:37 INFO KafkaConsumer: [Consumer clientId=consu

24/06/04 14:19:38 INFO DataWritingSparkTask: Writer for partition 1 is committing.
24/06/04 14:19:38 INFO DataWritingSparkTask: Writer for partition 0 is committing.
24/06/04 14:19:38 INFO DataWritingSparkTask: Committed partition 0 (task 0, attempt 0, stage 0.0)
24/06/04 14:19:38 INFO KafkaDataConsumer: From Kafka topicPartition=mytopic-0 groupId=spark-kafka-source-6e653502-7bdf-427e-ad03-8b6cb76d68c4--1298849648-executor read 1 records through 1 polls (polled  out 1 records), taking 664718426 nanos, during time span of 935378843 nanos.
24/06/04 14:19:38 INFO DataWritingSparkTask: Committed partition 1 (task 1, attempt 0, stage 0.0)
24/06/04 14:19:38 INFO KafkaDataConsumer: From Kafka topicPartition=mytopic-1 groupId=spark-kafka-source-6e653502-7bdf-427e-ad03-8b6cb76d68c4--1298849648-executor read 1 records through 1 polls (polled  out 1 records), taking 672170286 nanos, during time span of 938698445 nanos.
24/06/04 14:19:38 INFO CodeGenerator: Code generated in 33.276646 ms(0 + 2) / 

-------------------------------------------
Batch: 0
-------------------------------------------


...

24/06/04 15:23:02 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:23:03 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:23:12 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:23:13 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:23:22 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:23:23 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:23:32 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:23:33 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:23:42 INFO MicroBatchExecution: Stre

24/06/04 15:28:43 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:28:52 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:28:53 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:29:02 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:29:03 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:29:12 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:29:13 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:29:22 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:29:23 INFO MicroBatchExecution: Stre

24/06/04 15:34:32 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:34:33 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:34:42 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:34:43 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:34:52 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:34:53 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:35:02 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:35:03 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:35:12 INFO MicroBatchExecution: Stre

24/06/04 15:40:13 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:40:23 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:40:23 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:40:33 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:40:33 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:40:43 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:40:43 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:40:53 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:40:54 INFO MicroBatchExecution: Stre

24/06/04 15:46:03 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:46:04 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:46:13 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:46:14 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:46:23 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:46:24 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:46:33 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:46:34 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.
24/06/04 15:46:43 INFO MicroBatchExecution: Stre